In [2]:
import sys
import numpy as np
from os.path import abspath, join
sys.path.append(abspath(join('../..')))

from math import pi

from LevelSetPy.Utilities import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.Visualization import *
# from LevelSetPy.DynamicalSystems import *
from LevelSetPy.Grids import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration.Dissipation import *

# from LargeBRAT.BRATSolver.brt_solver import solve_brt

In [2]:

obj = Bundle({})

pdDims = 2; N = 100
v, w = +1, +1
obj.ve, obj.vp = v, v
obj.we, obj.wp = -w, w

In [3]:

# get player (pursuer) 1's state space
gmin = np.array(([[-5, -5, -pi]])).T
gmax = np.array(([[5, 5, pi]])).T
obj.p1 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p1.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.pursuer.center = 0

# get player (evader) 2's state space
gmin = np.array(([[-3, -3, -2*pi]])).T
gmax = np.array(([[7, 7, 2*pi]])).T
obj.p1.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.evader.center = np.array(([[1.5, 1.5, 4*pi/2]]),dtype=np.float64).T

# get player (pursuer) 3's state space
gmin = np.array(([[1, 1, 2]])).T
gmax = np.array(([[6, 6, 8]])).T
obj.p2 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p2.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.pursuer.center = np.array(([[3.5, 3.5, 4]]),dtype=np.float64).T

# get player (evader) 4's state space
gmin = np.array(([[3, 3, 2*pi/3]])).T
gmax = np.array(([[3*pi, 3*pi, 4*pi]])).T
obj.p2.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.evader.center = np.array(([[(3*pi-3)/2., (3*pi-3)/2., (4.0-2/3)*(pi/2.0)]]),dtype=np.float64).T



In [ ]:
p1, p2, mode= obj.p1.pursuer, obj.p1.evader, 'capture'

assert isfield(p1, 'center'), 'player I must have a center '\
                                'defined for its capture equation.'
assert isfield(p2, 'center'), 'player II must have a center '\
                                'defined for its capture equation.'
assert isfield(p1, 'grid'), 'player I must have its grid info'
assert isfield(p2, 'grid'), 'player II must have its grid info'

x1 = p1.grid.xs
x2 = p2.grid.xs

p1_dyn  = cell(3)
p2_dyn = cell(3)

p1_dyn[0] = obj.vp*np.cos(x1[2])
p1_dyn[1] = obj.vp*np.sin(x1[2])
p1_dyn[2] = obj.we

p2_dyn[0] = obj.ve*np.cos(x2[2])
p2_dyn[1] = obj.ve*np.sin(x2[2])
p2_dyn[2] = obj.wp

data_agent1 = np.zeros(p1.grid.shape)
data_agent2 = np.zeros(p2.grid.shape)

if isfield(p1, 'center') and numel(p1.center==1):
    p1.center = p1.center*np.ones((p1.grid.dim,1), dtype=np.float64)
if isfield(p2, 'center') and numel(p2.center==1):
    p2.center = p2.center*np.ones((p2.grid.dim,1), dtype=np.float64)
        

### This for the Double Integrator

In [13]:
import cupy as cp
import numpy as np

import sys
from os.path import abspath, join
sys.path.append(abspath(join('../..')))
from LevelSetPy.Utilities import *
from LevelSetPy.Grids import createGrid
from LevelSetPy.Helper import postTimeStepTTR
from LevelSetPy.Visualization import implicit_mesh
from LevelSetPy.DynamicalSystems import DoubleIntegrator
from LevelSetPy.SpatialDerivative import upwindFirstWENO5
from LevelSetPy.ExplicitIntegration import artificialDissipationGLF
from LevelSetPy.ExplicitIntegration.Integration import odeCFL3, odeCFLset
from LevelSetPy.ExplicitIntegration.Term import termRestrictUpdate, termLaxFriedrichs


In [5]:
gmin = np.array(([[-1, -1]]),dtype=np.float64).T
gmax = np.array(([[1, 1]]),dtype=np.float64).T
g = createGrid(gmin, gmax, 101, None)

eps_targ = 1.0
u_bound = 1
target_rad = .2 #eps_targ * np.max(g.dx)
dint = DoubleIntegrator(g, u_bound)
value_time2reach = dint.min_time2reach() - target_rad
value_time2reach = np.maximum(0, value_time2reach)

In [6]:
above_curve=dint.grid.xs[0]>dint.Gamma
below_curve=dint.grid.xs[0]<dint.Gamma
on_curve=dint.grid.xs[0]==dint.Gamma

reach_term1  = (dint.grid.xs[1] + np.emath.sqrt(4*dint.grid.xs[0] + \
                         2 * dint.grid.xs[1]**2))*above_curve
reach_term2 =  (-dint.grid.xs[1]+np.emath.sqrt(-4*dint.grid.xs[0] + \
                2 * dint.grid.xs[1]**2) )*below_curve
reach_term3 = np.abs(dint.grid.xs[1]) * on_curve
reach_time = reach_term1.real + reach_term2.real + reach_term3

In [16]:
dataIn = cp.arange(1, 13).reshape(3, 4)
print('dataIn\n', dataIn)
indices =[ [3], [5]]
print()
print(dataIn[cp.ix_(*indices)])

dataIn
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]

[[2]]


### Multiple vehicles

In [ ]:

obj = Bundle({})


pdDims = 2; N = 100
v, w = +1, +1
obj.ve, obj.vp = v, v
obj.we, obj.wp = -w, w


# get player (pursuer) 1's state space
gmin = np.array(([[-5, -5, -pi]])).T
gmax = np.array(([[0, 0, pi]])).T
obj.p1 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p1.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.pursuer.center = np.array(([[-2.5, -2.5, 0]]),dtype=np.float64).T
obj.p1.pursuer.radius = 0.5
obj.p1.pursuer.basis = np.array(([[1,0,0]]))

# get player (evader) 2's state space
gmin = np.array(([[0, 0, pi]])).T
gmax = np.array(([[5, 5, 3*pi]])).T
obj.p1.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.evader.center = np.array(([[2.5, 2.5, 2*pi]]),dtype=np.float64).T
obj.p1.evader.radius = .5
obj.p1.evader.basis = np.array(([[0,1,0]]))

# get player (pursuer) 3's state space
gmin = np.array(([[5, 5, 3*pi]])).T
gmax = np.array(([[10, 10, 5*pi]])).T
obj.p2 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p2.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.pursuer.center = np.array(([[7.5, 7.5, 4*pi]]),dtype=np.float64).T
obj.p2.pursuer.radius = .5
obj.p2.pursuer.basis = np.array(([[0,0,1]]))

# get player (evader) 4's state space
gmin = np.array(([[10, 10, 5*pi]])).T
gmax = np.array(([[15, 15, 7*pi]])).T
obj.p2.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.evader.center = np.array(([[12.5, 12.5, 6*pi]]),dtype=np.float64).T
obj.p2.evader.radius = .5
obj.p2.evader.basis = np.array(([[0,0,0,1]]))

# Full grid
gmin = np.array(([[-5, -5, -pi]])).T
gmax = np.array(([[15, 15, 7*pi]])).T
obj.full_grid = createGrid(gmin, gmax, N, pdDims)


In [ ]:
obj.p1.pursuer.xdot = dubins_absolute(obj, obj.p1.pursuer)
obj.p1.evader.xdot  = dubins_absolute(obj, obj.p1.evader)
obj.p2.pursuer.xdot = dubins_absolute(obj, obj.p2.pursuer)
obj.p2.evader.xdot  = dubins_absolute(obj, obj.p2.evader)
 
value_func = shapeRectangleByCorners(obj.full_grid, lower=-3, upper=13)

In [ ]:
# we now have a large value function, decompose the value w.r.t to the
# basis of the four vehicles to get its correspondiung decomposition into diff bases

In [ ]:
# compose rhe full grid as a tensor of all players
obj.grid_compose_vs = obj.p1.pursuer.grid.vs + obj.p1.evader.grid.vs + \
                      obj.p2.pursuer.grid.vs + obj.p2.evader.grid.vs
print([x.shape for x in obj.grid_compose_vs])

# too big
#FullTensor = np.meshgrid(*obj.grid_compose_vs, indexing='ij')

In [ ]:
# direction cosine of p1.pursuer w.r.t p1.evader
obj.p1.pursuer.basis_p1evader = np.dot(obj.p1.pursuer.basis.T, obj.p1.evader.basis)

In [ ]:
p1_pursuer = np.asarray(obj.p1.pursuer.grid.xs).transpose([1,2,3,0])
p1_evader = np.asarray(obj.p1.evader.grid.xs).transpose([1,2,3,0])
p2_pursuer = np.asarray(obj.p2.pursuer.grid.xs).transpose([1,2,3,0])
p2_evader = np.asarray(obj.p2.evader.grid.xs).transpose([1,2,3,0])
FT = np.concatenate((p1_pursuer, p1_evader), )
FT.shape

# use HOSVD to compute optimal rotation vector of one agent's state space to another's state space

def kabsch(P, Q):
    """
    Using the Kabsch algorithm with two sets of paired point P and Q, centered
    around the centroid. Each vector set is represented as an NxD
    matrix, where D is the the dimension of the space.
    The algorithm works in three steps:
    - a centroid translation of P and Q (assumed done before this function
      call)
    - the computation of a covariance matrix C
    - computation of the optimal rotation matrix U
    For more info see http://en.wikipedia.org/wiki/Kabsch_algorithm
    Parameters
    ----------
    P : array
        (N,D) matrix, where N is points and D is dimension.
    Q : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    U : matrix
        Rotation matrix (D,D)
    """

    # Computation of the covariance matrix
    C = np.dot(np.transpose(P), Q)

    # Computation of the optimal rotation matrix
    # This can be done using singular value decomposition (SVD)
    # Getting the sign of the det(V)*(W) to decide
    # whether we need to correct our rotation matrix to ensure a
    # right-handed coordinate system.
    # And finally calculating the optimal rotation matrix U
    # see http://en.wikipedia.org/wiki/Kabsch_algorithm
    V, S, W = np.linalg.svd(C)
    d = (np.linalg.det(V) * np.linalg.det(W)) < 0.0

    if d:
        S[-1] = -S[-1]
        V[:, -1] = -V[:, -1]

    # Create Rotation matrix U
    U = np.dot(V, W)

    return U


In [72]:
import rmsd

# translate p1_pursuer into p1_evader's state space
p1_pursuer -= p2_evader

# transform the state space pf p1 pursuer into p1 evader
p1_pursuer-=rmsd.centroid(p1_pursuer)
p1_evader-=rmsd.centroid(p1_evader)

# get the optimal rotation matrices between the grid points
U = rmsd.kabsch(p1_pursuer, p1_evader)
p1_pursuer_in_evader = np.dot(p1_pursuer, U)


MemoryError: Unable to allocate 224. GiB for an array with shape (3, 100, 100, 100, 100, 100) and data type float64